<a href="https://colab.research.google.com/github/Jonathanpro/myaiblog/blob/master/_notebooks/2021-07-10-huggingface_ner_pytorch_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP -  Named Entity Recognition - Inference
> A tutorial for appyling inference to a named entity recognition with Huggingface & Pytroch.

- toc: true 
- badges: true
- comments: false
- categories: [jupyter, NLP]
- image: images/chart-preview.png


In [1]:
!pip install transformers

     |████████████████████████████████| 2.5MB 4.1MB/s 
     |████████████████████████████████| 3.3MB 25.0MB/s 
     |████████████████████████████████| 901kB 43.0MB/s 


In [2]:
!pip install spacy -U

     |████████████████████████████████| 6.4MB 5.1MB/s 
     |████████████████████████████████| 51kB 8.4MB/s 
     |████████████████████████████████| 624kB 40.0MB/s 
     |████████████████████████████████| 10.1MB 32.1MB/s 
     |████████████████████████████████| 460kB 38.7MB/s 
  Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [9]:
!curl https://github.com/elenanereiss/Legal-Entity-Recognition/raw/master/data/dataset_courts.zip -L -o /content/raw.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   168  100   168    0     0   1322      0 --:--:-- --:--:-- --:--:--  1312
100 4289k  100 4289k    0     0  14.6M      0 --:--:-- --:--:-- --:--:-- 14.6M


In [11]:
ls /content/

gdrive/  raw.zip  sample_data/


In [12]:
!unzip /content/raw.zip -d /content/raw/

Archive:  /content/raw.zip
  inflating: /content/raw/bfh.conll  
  inflating: /content/raw/bgh.conll  
  inflating: /content/raw/bpatg.conll  
  inflating: /content/raw/bsg.conll  
  inflating: /content/raw/bverfg.conll  
  inflating: /content/raw/bverwg.conll  
  inflating: /content/raw/bag.conll  


In [13]:
from pathlib import Path
import re

def read_wnut(file_path):
    file_path = Path(file_path)

    raw_text = file_path.read_text().strip()
    raw_docs = re.split(r'\n\t?\n', raw_text)
    token_docs = []
    tag_docs = []
    for doc in raw_docs:
        tokens = []
        tags = []
        for line in doc.split('\n'):
            # print(line)
            token, tag = line.split()
            tokens.append(token)
            tags.append(tag)
        token_docs.append(tokens)
        tag_docs.append(tags)

    return token_docs, tag_docs

train_texts, train_tags = read_wnut('/content/raw/bag.conll')
val_texts, val_tags = read_wnut('/content/raw/bgh.conll')

In [14]:
tags = train_tags + val_tags

In [15]:
unique_tags = set(tag for doc in tags for tag in doc)
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

In [16]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=10,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=1000,
)

from transformers import AutoModelForTokenClassification
model = AutoModelForTokenClassification.from_pretrained("/content/gdrive/MyDrive/NER_01_030_2021_07_08.bin", local_files_only=True, num_labels=len(unique_tags))

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    # train_dataset=train_dataset,         # training dataset
    # eval_dataset=val_dataset             # evaluation dataset
)

In [17]:
import torch

class COURTS_Dataset_inf(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

In [18]:
from transformers import AutoTokenizer
MODEL_NAME = 'bert-base-german-cased' 
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

https://huggingface.co/bert-base-german-cased/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpu0r451i2


storing https://huggingface.co/bert-base-german-cased/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/2529d64cc99a539f2103ad09cea0d6459e181d8dc168fe06b32d25ddc68e6d3b.ec5c189f89475aac7d8cbd243960a0655cfadc3d0474da8ff2ed0bf1699c2a5f
creating metadata file for /root/.cache/huggingface/transformers/2529d64cc99a539f2103ad09cea0d6459e181d8dc168fe06b32d25ddc68e6d3b.ec5c189f89475aac7d8cbd243960a0655cfadc3d0474da8ff2ed0bf1699c2a5f


https://huggingface.co/bert-base-german-cased/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp17luf07x


storing https://huggingface.co/bert-base-german-cased/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/98877e98ee76b3977d326fe4f54bc29f10b486c317a70b6445ac19a0603b00f0.1f2afedb22f9784795ae3a26fe20713637c93f50e2c99101d952ea6476087e5e
creating metadata file for /root/.cache/huggingface/transformers/98877e98ee76b3977d326fe4f54bc29f10b486c317a70b6445ac19a0603b00f0.1f2afedb22f9784795ae3a26fe20713637c93f50e2c99101d952ea6476087e5e
loading configuration file https://huggingface.co/bert-base-german-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/98877e98ee76b3977d326fe4f54bc29f10b486c317a70b6445ac19a0603b00f0.1f2afedb22f9784795ae3a26fe20713637c93f50e2c99101d952ea6476087e5e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermedia

https://huggingface.co/bert-base-german-cased/resolve/main/vocab.txt not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpgtu9z249


storing https://huggingface.co/bert-base-german-cased/resolve/main/vocab.txt in cache at /root/.cache/huggingface/transformers/0c57cb5172c1ac6c957d00597dc43c1b8b2a2cb44729a590fd0112612221f746.9a4f439638381be22bb9f116542bdaa5e1d8bb7a09a5f8ef32d9662deaf655a1
creating metadata file for /root/.cache/huggingface/transformers/0c57cb5172c1ac6c957d00597dc43c1b8b2a2cb44729a590fd0112612221f746.9a4f439638381be22bb9f116542bdaa5e1d8bb7a09a5f8ef32d9662deaf655a1


https://huggingface.co/bert-base-german-cased/resolve/main/tokenizer.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp7ssc0k_c


storing https://huggingface.co/bert-base-german-cased/resolve/main/tokenizer.json in cache at /root/.cache/huggingface/transformers/a60c7a72be0cad1606096bd88aa22980c826a10b2482a850cfd50db5ceb3f01f.a1d3fa1580dc5318a8ad0477d679498575453bbe1ef5751aaca7fec558055f77
creating metadata file for /root/.cache/huggingface/transformers/a60c7a72be0cad1606096bd88aa22980c826a10b2482a850cfd50db5ceb3f01f.a1d3fa1580dc5318a8ad0477d679498575453bbe1ef5751aaca7fec558055f77


loading file https://huggingface.co/bert-base-german-cased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/0c57cb5172c1ac6c957d00597dc43c1b8b2a2cb44729a590fd0112612221f746.9a4f439638381be22bb9f116542bdaa5e1d8bb7a09a5f8ef32d9662deaf655a1
loading file https://huggingface.co/bert-base-german-cased/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/a60c7a72be0cad1606096bd88aa22980c826a10b2482a850cfd50db5ceb3f01f.a1d3fa1580dc5318a8ad0477d679498575453bbe1ef5751aaca7fec558055f77
loading file https://huggingface.co/bert-base-german-cased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-german-cased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-german-cased/resolve/main/tokenizer_config.json from cache at /root/.cache/huggingface/transformers/2529d64cc99a539f2103ad09cea0d6459e181d8dc168fe06b32d25ddc68e6d3b.ec5c189f89475aac7d8cbd243960a065

In [19]:
def predict_text(text, train_tags=train_tags,  trainer=trainer, tokenizer=tokenizer):
    train_encodings = tokenizer([text], is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)
    train_encodings.pop("offset_mapping") # we don't want to pass this to the model
    train_dataset = COURTS_Dataset_inf(train_encodings)
    result = trainer.predict(train_dataset)

    token_ids = tokenizer.encode(text[0], add_special_tokens=False)
    word_tag_dic = []
    preds = np.argmax(result.predictions, axis=2)
    x=1
    for word in text[0].split(' '):
        word_tag_dic.append([word, id2tag[preds[0][x]]])
        x=x+1
    return word_tag_dic

In [20]:
import numpy as np

In [21]:
word_tag_dic=predict_text([' '.join(train_texts[3])])

***** Running Prediction *****
  Num examples = 1
  Batch size = 64


In [22]:
word_tag_dic

[['Die', 'B-RS'],
 ['beabsichtigte', 'B-RS'],
 ['Klage', 'B-RS'],
 ['auf', 'B-RS'],
 ['Entschädigung', 'B-RS'],
 ['wegen', 'B-RS'],
 ['der', 'B-RS'],
 ['Dauer', 'B-RS'],
 ['des', 'B-RS'],
 ['Verfahrens', 'I-INN'],
 ['vor', 'B-LIT'],
 ['dem', 'B-LIT'],
 ['Bundesarbeitsgericht', 'B-LIT'],
 ['zum', 'B-LIT'],
 ['Aktenzeichen', 'B-LIT'],
 ['-', 'B-LIT'],
 ['8', 'B-LIT'],
 ['AZR', 'B-LIT'],
 ['418/15', 'B-LIT'],
 ['-', 'B-LIT'],
 ['bietet', 'B-LIT'],
 ['keine', 'B-LIT'],
 ['hinreichende', 'B-LIT'],
 ['Aussicht', 'B-LIT'],
 ['auf', 'B-LIT'],
 ['Erfolg', 'B-RS'],
 [',', 'B-RS'],
 ['§', 'B-RS'],
 ['114', 'B-RS'],
 ['Abs.', 'B-RS'],
 ['1', 'B-RS'],
 ['Satz', 'B-RS'],
 ['1', 'B-LDS'],
 ['ZPO', 'B-VT'],
 ['.', 'B-VT']]

In [23]:
from spacy.vocab import Vocab
from spacy.tokens.doc import Doc
from spacy.tokens import Span

In [24]:
def predict_and_visulize(text, train_tags=train_tags,  trainer=trainer, tokenizer=tokenizer):
    word_tag_dic=predict_text([' '.join(text)])
    x=0
    span_list = []
    vocab = Vocab(strings=text)
    doc = Doc(vocab, words=text)
    for token in word_tag_dic:
      # print(x)
      span_list.append(Span(doc, x, x+1, token[1]))
      x=x+1
    doc.set_ents(span_list)
    return doc

In [27]:
from spacy import displacy
doc = predict_and_visulize(train_texts[6])
displacy.render(doc , jupyter=True, style='ent')

***** Running Prediction *****
  Num examples = 1
  Batch size = 64
